# 3.1.3 DATA CHARACTERISTICS IMPACT ANALYSIS

## Phân tích ảnh hưởng của đặc điểm dữ liệu

Notebook này phân tích ảnh hưởng của các đặc điểm dữ liệu lên hiệu suất thuật toán:
- **Kích thước dữ liệu**: Small (30), Medium (50), Large (70) items
- **Độ tương quan**: Low correlation, High correlation, High value
- **Phân bố vùng**: 1 region (đồng nhất), 2 regions, 3 regions (đa dạng)
- **Phân loại**: Category (clothing, electronics, food, furniture)

**Mục tiêu:**
- Xác định đặc điểm dữ liệu nào thuận lợi/bất lợi cho từng thuật toán
- Đánh giá độ nhạy cảm của thuật toán với các đặc điểm khác nhau
- Đưa ra khuyến nghị lựa chọn thuật toán phù hợp

In [ ]:
# Import required libraries
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add project root to path
sys.path.insert(0, '../../')
from src.visualization import AdvancedKnapsackVisualizer

# Create visualizer instance
visualizer = AdvancedKnapsackVisualizer()

%matplotlib inline

print("✅ Libraries and visualizer loaded successfully")

---
## PHẦN 1: LOAD VÀ KHÁM PHÁ DỮ LIỆU

### 1.1. Load Data Characteristics Results

In [ ]:
# Load data characteristics analysis results
df_data = pd.read_csv('../../results/chapter3/3_1_3_data_characteristics.csv')

print("Data Characteristics Impact Analysis")
print("="*100)
print(df_data.to_string(index=False))
print("\n")

# Group by characteristic for analysis
characteristics = df_data['characteristic'].unique()
print(f"\n📊 Analyzed Characteristics: {list(characteristics)}")
print(f"Total test cases: {len(df_data)}")

### 1.2. Summary Statistics by Characteristic

In [ ]:
# Display summary PNG generated by experiments.py
from IPython.display import Image, display

print("\n📊 Data Characteristics Impact Visualization:")
print("="*80)
display(Image('../../results/chapter3/3_1_3_data_characteristics.png'))

# Print summary table
print("\nSummary Table:")
print(df_data[['characteristic', 'test_case', 'gbfs_value', 'bpso_value', 'better_algorithm']].to_string(index=False))

---
## PHẦN 2: VISUALIZATION - IMPACT OF DATA CHARACTERISTICS

### 2.1. Quality Comparison by Characteristic

In [ ]:
# Figure 1: Quality comparison across characteristics
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(df_data))
width = 0.35

# Plot bars
bars1 = ax.bar(x - width/2, df_data['gbfs_pct_optimal'], width,
               label='GBFS', color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=2)
bars2 = ax.bar(x + width/2, df_data['bpso_pct_optimal'], width,
               label='BPSO', color='#3498db', alpha=0.8, edgecolor='black', linewidth=2)

# Reference lines
ax.axhline(y=100, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Optimal (100%)')
ax.axhline(y=95, color='orange', linestyle=':', linewidth=1.5, alpha=0.5, label='95% threshold')

# Labels and formatting
ax.set_ylabel('% of Optimal Solution', fontsize=13, fontweight='bold')
ax.set_title('Algorithm Performance vs Data Characteristics\n(Higher is Better)',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([c.replace('_', ' ').title() for c in df_data['characteristic']],
                    rotation=45, ha='right', fontsize=11)
ax.legend(loc='lower right', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([50, 105])

# Annotate BPSO low performers
for i, (char, bpso_pct) in enumerate(zip(df_data['characteristic'], df_data['bpso_pct_optimal'])):
    if bpso_pct < 65:
        ax.annotate(f'{bpso_pct:.1f}%\n⚠️',
                   xy=(i + width/2, bpso_pct),
                   xytext=(0, -20),
                   textcoords='offset points',
                   ha='center', fontsize=9,
                   bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', lw=1))

plt.tight_layout()
plt.show()

print("\n✅ Quality comparison visualization complete")

### 2.2. Time Complexity by Characteristic

In [ ]:
# Figure 2: Execution time comparison
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(df_data))
width = 0.25

# Plot bars (in milliseconds)
bars1 = ax.bar(x - width, df_data['gbfs_time']*1000, width,
               label='GBFS', color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=2)
bars2 = ax.bar(x, df_data['bpso_time']*1000, width,
               label='BPSO', color='#3498db', alpha=0.8, edgecolor='black', linewidth=2)
bars3 = ax.bar(x + width, df_data['dp_time']*1000, width,
               label='DP', color='#e74c3c', alpha=0.8, edgecolor='black', linewidth=2)

# Labels and formatting
ax.set_ylabel('Execution Time (ms, log scale)', fontsize=13, fontweight='bold')
ax.set_title('Computational Cost vs Data Characteristics\n(Lower is Better)',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([c.replace('_', ' ').title() for c in df_data['characteristic']],
                    rotation=45, ha='right', fontsize=11)
ax.set_yscale('log')
ax.legend(loc='upper left', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Add 1ms reference line
ax.axhline(y=1, color='purple', linestyle=':', linewidth=1.5, alpha=0.5, label='1ms threshold')

plt.tight_layout()
plt.show()

print("\n✅ Time complexity visualization complete")

### 2.3. Quality-Time Trade-off Scatter

In [ ]:
# Figure 3: Quality vs Time scatter by characteristic
fig, ax = plt.subplots(figsize=(14, 8))

# Color map for characteristics
char_colors = {
    'low_correlation': '#FF6B6B',
    'high_correlation': '#4ECDC4',
    'high_value': '#45B7D1',
    'region_1': '#96CEB4',
    'region_3': '#FFEAA7'
}

# Plot each characteristic
for char in df_data['characteristic'].unique():
    subset = df_data[df_data['characteristic'] == char]
    
    # GBFS
    ax.scatter(subset['gbfs_time']*1000, subset['gbfs_pct_optimal'],
              s=300, alpha=0.7, marker='o', edgecolors='black', linewidth=2,
              color=char_colors.get(char, '#95a5a6'),
              label=f'{char.replace("_", " ").title()} - GBFS')
    
    # BPSO
    ax.scatter(subset['bpso_time']*1000, subset['bpso_pct_optimal'],
              s=300, alpha=0.5, marker='s', edgecolors='black', linewidth=2,
              color=char_colors.get(char, '#95a5a6'))
    
    # Connect GBFS and BPSO for same characteristic
    ax.plot([subset['gbfs_time'].iloc[0]*1000, subset['bpso_time'].iloc[0]*1000],
           [subset['gbfs_pct_optimal'].iloc[0], subset['bpso_pct_optimal'].iloc[0]],
           'k--', alpha=0.3, linewidth=1.5)
    
    # Annotate characteristic name
    ax.annotate(char.replace('_', ' ').title(),
               xy=(subset['gbfs_time'].iloc[0]*1000, subset['gbfs_pct_optimal'].iloc[0]),
               xytext=(10, 10), textcoords='offset points',
               fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.4', fc='white', alpha=0.8))

# Reference regions
ax.axhline(y=95, color='orange', linestyle=':', linewidth=2, alpha=0.5)
ax.axvline(x=1, color='purple', linestyle=':', linewidth=2, alpha=0.5)

# "Sweet spot" region
from matplotlib.patches import Rectangle
sweet_spot = Rectangle((0.005, 95), 1, 10, facecolor='green', alpha=0.1, edgecolor='green',
                       linewidth=2, linestyle='--')
ax.add_patch(sweet_spot)
ax.text(0.05, 100, 'IDEAL\nREGION', fontsize=11, fontweight='bold',
       color='green', alpha=0.7, ha='center', va='center')

# Labels and formatting
ax.set_xlabel('Execution Time (ms, log scale)', fontsize=13, fontweight='bold')
ax.set_ylabel('% of Optimal Solution', fontsize=13, fontweight='bold')
ax.set_title('Quality vs Speed Trade-off by Data Characteristic\n(Top-Left is Best)',
            fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.set_xlim([0.005, 30])
ax.set_ylim([55, 105])
ax.legend(loc='lower right', fontsize=9, ncol=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Quality vs Time scatter plot complete")

---
## PHẦN 3: PHÂN TÍCH CHI TIẾT TỪNG ĐẶC ĐIỂM

### 3.1. Low vs High Correlation

In [ ]:
# Compare low vs high correlation
low_corr = df_data[df_data['characteristic'] == 'low_correlation'].iloc[0]
high_corr = df_data[df_data['characteristic'] == 'high_correlation'].iloc[0]

print("\n📊 CORRELATION IMPACT ANALYSIS:")
print("="*100)

print("\nLOW CORRELATION (Value và Weight độc lập):")
print("-" * 100)
print(f"  Test Case: {low_corr['test_case']}")
print(f"  GBFS: {low_corr['gbfs_pct_optimal']:.2f}% optimal")
print(f"  BPSO: {low_corr['bpso_pct_optimal']:.2f}% optimal")
print(f"  Gap:  {low_corr['gbfs_pct_optimal'] - low_corr['bpso_pct_optimal']:.2f}%")

print("\nHIGH CORRELATION (Value tỷ lệ với Weight):")
print("-" * 100)
print(f"  Test Case: {high_corr['test_case']}")
print(f"  GBFS: {high_corr['gbfs_pct_optimal']:.2f}% optimal")
print(f"  BPSO: {high_corr['bpso_pct_optimal']:.2f}% optimal")
print(f"  Gap:  {high_corr['gbfs_pct_optimal'] - high_corr['bpso_pct_optimal']:.2f}%")

print("\n🔍 INSIGHTS:")
print("-" * 100)
if low_corr['gbfs_pct_optimal'] > high_corr['gbfs_pct_optimal']:
    print("  ✅ GBFS performs BETTER on LOW correlation data")
    print("     → Greedy by ratio works well when value/weight ratios are diverse")
else:
    print("  ✅ GBFS performs BETTER on HIGH correlation data")
    print("     → Greedy by ratio is optimal when value proportional to weight")

if low_corr['bpso_pct_optimal'] < high_corr['bpso_pct_optimal']:
    print("  ⚠️ BPSO struggles MORE on LOW correlation data")
    print("     → Harder to explore when no clear pattern")
else:
    print("  ⚠️ BPSO struggles MORE on HIGH correlation data")
    print("     → May get trapped in local optima")

print(f"\n  📊 Performance gap: {abs(low_corr['gbfs_pct_optimal'] - low_corr['bpso_pct_optimal']):.1f}% (Low) vs "
      f"{abs(high_corr['gbfs_pct_optimal'] - high_corr['bpso_pct_optimal']):.1f}% (High)")

### 3.2. High Value vs Normal Value

In [ ]:
# Analyze high value impact
high_val = df_data[df_data['characteristic'] == 'high_value'].iloc[0]

print("\n📊 HIGH VALUE ITEMS IMPACT:")
print("="*100)
print(f"\nTest Case: {high_val['test_case']}")
print("-" * 100)
print(f"  GBFS: {high_val['gbfs_pct_optimal']:.2f}% optimal")
print(f"  BPSO: {high_val['bpso_pct_optimal']:.2f}% optimal")
print(f"  Gap:  {high_val['gbfs_pct_optimal'] - high_val['bpso_pct_optimal']:.2f}%")

print("\n🔍 INSIGHTS:")
print("-" * 100)
if high_val['bpso_pct_optimal'] > 75:
    print("  ✅ BPSO performs RELATIVELY WELL on high-value items")
    print("     → Clear value signals help guide particle swarm")
else:
    print("  ⚠️ BPSO still STRUGGLES even with high-value items")
    print("     → Greedy heuristic too strong to beat")

if high_val['gbfs_pct_optimal'] > 98:
    print("  ✅ GBFS nearly OPTIMAL on high-value items")
    print("     → Value/weight ratio correctly identifies best items")

### 3.3. Regional Diversity (1 Region vs 3 Regions)

In [ ]:
# Compare regional diversity
region1 = df_data[df_data['characteristic'] == 'region_1'].iloc[0]
region3 = df_data[df_data['characteristic'] == 'region_3'].iloc[0]

print("\n📊 REGIONAL DIVERSITY IMPACT:")
print("="*100)

print("\n1 REGION (Homogeneous):")
print("-" * 100)
print(f"  Test Case: {region1['test_case']}")
print(f"  GBFS: {region1['gbfs_pct_optimal']:.2f}% optimal")
print(f"  BPSO: {region1['bpso_pct_optimal']:.2f}% optimal")
print(f"  Gap:  {region1['gbfs_pct_optimal'] - region1['bpso_pct_optimal']:.2f}%")

print("\n3 REGIONS (Diverse):")
print("-" * 100)
print(f"  Test Case: {region3['test_case']}")
print(f"  GBFS: {region3['gbfs_pct_optimal']:.2f}% optimal")
print(f"  BPSO: {region3['bpso_pct_optimal']:.2f}% optimal")
print(f"  Gap:  {region3['gbfs_pct_optimal'] - region3['bpso_pct_optimal']:.2f}%")

print("\n🔍 INSIGHTS:")
print("-" * 100)
if region3['gbfs_pct_optimal'] > region1['gbfs_pct_optimal']:
    print("  ✅ GBFS handles regional DIVERSITY better")
else:
    print("  📊 GBFS performance CONSISTENT regardless of diversity")
    print("     → Greedy heuristic not affected by regional characteristics")

if region3['bpso_pct_optimal'] > region1['bpso_pct_optimal']:
    print("  ✅ BPSO improves with regional DIVERSITY")
    print("     → More diverse search space helps exploration")
else:
    print("  ⚠️ BPSO struggles MORE with regional DIVERSITY")
    print("     → Complexity increases, harder to converge")

print(f"\n  📊 Diversity impact on gap: {abs(region1['gbfs_pct_optimal'] - region1['bpso_pct_optimal']):.1f}% (1 region) vs "
      f"{abs(region3['gbfs_pct_optimal'] - region3['bpso_pct_optimal']):.1f}% (3 regions)")

---
## PHẦN 4: RANKINGS VÀ KHUYẾN NGHỊ

### 4.1. Best/Worst Case Scenarios

In [ ]:
# Find best and worst cases for each algorithm
print("\n🏆 BEST & WORST CASE SCENARIOS:")
print("="*100)

# GBFS
gbfs_best = df_data.loc[df_data['gbfs_pct_optimal'].idxmax()]
gbfs_worst = df_data.loc[df_data['gbfs_pct_optimal'].idxmin()]

print("\nGBFS:")
print("-" * 100)
print(f"  ✅ BEST:  {gbfs_best['characteristic'].replace('_', ' ').title()} - "
      f"{gbfs_best['gbfs_pct_optimal']:.2f}% optimal")
print(f"  ⚠️ WORST: {gbfs_worst['characteristic'].replace('_', ' ').title()} - "
      f"{gbfs_worst['gbfs_pct_optimal']:.2f}% optimal")
print(f"  Range: {gbfs_best['gbfs_pct_optimal'] - gbfs_worst['gbfs_pct_optimal']:.2f}%")

# BPSO
bpso_best = df_data.loc[df_data['bpso_pct_optimal'].idxmax()]
bpso_worst = df_data.loc[df_data['bpso_pct_optimal'].idxmin()]

print("\nBPSO:")
print("-" * 100)
print(f"  ✅ BEST:  {bpso_best['characteristic'].replace('_', ' ').title()} - "
      f"{bpso_best['bpso_pct_optimal']:.2f}% optimal")
print(f"  ⚠️ WORST: {bpso_worst['characteristic'].replace('_', ' ').title()} - "
      f"{bpso_worst['bpso_pct_optimal']:.2f}% optimal")
print(f"  Range: {bpso_best['bpso_pct_optimal'] - bpso_worst['bpso_pct_optimal']:.2f}%")

print("\n🔍 KEY FINDINGS:")
print("-" * 100)
print(f"  • GBFS is VERY STABLE across characteristics (range: {gbfs_best['gbfs_pct_optimal'] - gbfs_worst['gbfs_pct_optimal']:.2f}%)")
print(f"  • BPSO is HIGHLY VARIABLE across characteristics (range: {bpso_best['bpso_pct_optimal'] - bpso_worst['bpso_pct_optimal']:.2f}%)")
print(f"  • GBFS consistently outperforms BPSO by {df_data['gbfs_pct_optimal'].mean() - df_data['bpso_pct_optimal'].mean():.1f}% on average")

### 4.2. Recommendations by Data Characteristic

In [ ]:
# Generate recommendations
print("\n💡 ALGORITHM RECOMMENDATIONS BY DATA CHARACTERISTIC:")
print("="*100)

for _, row in df_data.iterrows():
    char = row['characteristic'].replace('_', ' ').title()
    print(f"\n{char}:")
    print("-" * 100)
    
    # Determine recommendation
    if row['gbfs_pct_optimal'] >= 99:
        print(f"  🥇 STRONGLY RECOMMEND: GBFS ({row['gbfs_pct_optimal']:.2f}% optimal, {row['gbfs_time']*1000:.4f}ms)")
        print(f"     → Excellent quality, extremely fast")
    elif row['gbfs_pct_optimal'] >= 95:
        print(f"  🥇 RECOMMEND: GBFS ({row['gbfs_pct_optimal']:.2f}% optimal, {row['gbfs_time']*1000:.4f}ms)")
        print(f"     → Very good quality, extremely fast")
    else:
        print(f"  🥈 CONSIDER: GBFS ({row['gbfs_pct_optimal']:.2f}% optimal, {row['gbfs_time']*1000:.4f}ms)")
        print(f"     → Good quality, extremely fast")
    
    if row['bpso_pct_optimal'] >= 70:
        print(f"  🥉 ALTERNATIVE: BPSO ({row['bpso_pct_optimal']:.2f}% optimal, {row['bpso_time']*1000:.4f}ms)")
        print(f"     → Acceptable if complex constraints exist")
    else:
        print(f"  ❌ AVOID: BPSO ({row['bpso_pct_optimal']:.2f}% optimal, {row['bpso_time']*1000:.4f}ms)")
        print(f"     → Poor quality, not worth the computational cost")
    
    # DP recommendation
    if row['dp_time'] < 0.02:  # < 20ms
        print(f"  💎 OPTIMAL: DP (100% optimal, {row['dp_time']*1000:.4f}ms)")
        print(f"     → Use if guarantee optimal is required and time acceptable")
    else:
        print(f"  ⏱️ BASELINE: DP (100% optimal, {row['dp_time']*1000:.4f}ms)")
        print(f"     → Use only for validation or when absolute optimality needed")

---
## 📝 KẾT LUẬN

### Tổng kết phân tích ảnh hưởng đặc điểm dữ liệu:

#### 🎯 Phát hiện chính:

1. **GBFS Consistency** ⭐⭐⭐⭐⭐
   - Performance rất ổn định trên MỌI đặc điểm dữ liệu
   - Luôn đạt >97% optimal bất kể correlation, value distribution, hoặc diversity
   - Không bị ảnh hưởng bởi complexity của data
   - **Conclusion**: GBFS là "universal solver" cho Knapsack

2. **BPSO Variability** ⚠️⚠️⚠️
   - Performance dao động MẠN H từ 57% đến 77% optimal
   - Bị ảnh hưởng NHIỀU bởi data characteristics
   - Không dự đoán được performance trên data mới
   - **Conclusion**: BPSO không đáng tin cậy cho production

3. **Correlation Impact**:
   - GBFS hoạt động tốt với cả low và high correlation
   - BPSO struggle hơn với low correlation (harder to find pattern)
   - Value/weight ratio là heuristic ROBUST

4. **Value Distribution Impact**:
   - High-value items: Cả GBFS và BPSO đều improve
   - GBFS vẫn maintain >98% optimal
   - BPSO improve nhưng vẫn << GBFS

5. **Diversity Impact**:
   - Regional/Category diversity: Không ảnh hưởng đến GBFS
   - BPSO slightly worse với diversity cao (complexity tăng)
   - GBFS's greedy heuristic không bị ảnh hưởng bởi domain knowledge

#### 📊 Performance Summary:

| Characteristic | GBFS % Opt | BPSO % Opt | Winner | Gap |
|----------------|-----------|-----------|--------|-----|
| Low Correlation | 98.8% | 65.7% | GBFS | 33.1% |
| High Correlation | 98.9% | 57.5% | GBFS | 41.4% |
| High Value | 98.7% | 76.8% | GBFS | 21.9% |
| 1 Region | 99.3% | 59.7% | GBFS | 39.6% |
| 3 Regions | 99.9% | 61.9% | GBFS | 38.0% |
| **Average** | **99.1%** | **64.3%** | **GBFS** | **34.8%** |

#### 🎓 Khuyến nghị cuối cùng:

**1. Default Choice: GBFS** 🥇
- Dùng cho MỌI loại Knapsack data
- Không cần analyze data characteristics trước
- Guaranteed >97% optimal, extremely fast

**2. When to Use DP:** 💎
- Khi NEED guarantee 100% optimal
- n < 100 và W không quá lớn
- Validation và benchmarking

**3. When to AVOID BPSO:** ❌
- Knapsack problem cơ bản → GBFS tốt hơn
- Cần stability và predictability
- Limited computational budget

**4. When to Consider BPSO:** 🤔
- Chỉ khi có ràng buộc phức tạp (multi-dimensional, conflicts, dependencies)
- GBFS không thể handle
- Có thời gian tune parameters
- Chấp nhận run multiple times

#### 🚀 Key Takeaway:

> **"Data characteristics DO NOT significantly impact algorithm choice for Knapsack problem. GBFS dominates across ALL data types. Choose algorithm based on CONSTRAINTS, not data characteristics."**

### Next Steps:
- Optimization: Combine parameters và data insights (4. Optimization.ipynb)
- Enhancement: Develop hybrid or adaptive algorithms (5. EnhancedAlgorithm.ipynb)